In [2]:
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import numpy as np
import evaluate
import datasets
import spacy
import tqdm

# Tokenizers
import re
import unicodedata
import jieba

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using", device, "device")

c:\Users\user\OneDrive\Documents\GitHub\cs4248-neural-translation\venvE2C\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda device


In [3]:
dataset = load_dataset("iwslt2017", "iwslt2017-en-zh") 
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 231266
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 8549
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 879
    })
})

In [4]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

train_data

Dataset({
    features: ['translation'],
    num_rows: 231266
})

In [5]:
torch.cuda.empty_cache()

## Tokenizers

In [6]:
spacy.prefer_gpu() # require_gpu raises an error if no GPU is avail 
nlp = spacy.load("en_core_web_md")

In [7]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

## Tokenize en and zh words

In [8]:
def tokenize_pair(pair, max_length, sos_token, eos_token):
    en_tokens = [token.text for token in nlp.tokenizer(normalizeString(pair['translation']['en']))][:max_length]
    zh_tokens = list(jieba.cut(pair['translation']['zh'], cut_all=False))[:max_length] # if cut_all is True, it will use the full mode. Default is accurate mode.
    en_tokens = [sos_token] + en_tokens +  [eos_token]
    zh_tokens = [sos_token] + zh_tokens +  [eos_token]
    return {"en_tokens": en_tokens, "zh_tokens": zh_tokens}

In [9]:
# Additional arguments to the func above needs to be stored in a dictionary and passed to fn_kwargs arg of map 

max_length = 20
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "max_length": max_length,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_tokens = train_data.map(tokenize_pair, fn_kwargs=fn_kwargs)
valid_tokens = valid_data.map(tokenize_pair, fn_kwargs=fn_kwargs)
test_tokens = test_data.map(tokenize_pair, fn_kwargs=fn_kwargs)


Map:   0%|          | 0/879 [00:00<?, ? examples/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.478 seconds.
Prefix dict has been built successfully.
Map: 100%|██████████| 8549/8549 [00:02<00:00, 4105.29 examples/s]


## Vocabularies

In [13]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_tokens["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

zh_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_tokens["zh_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [19]:
assert en_vocab[unk_token] == zh_vocab[unk_token]
assert en_vocab[pad_token] == zh_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [20]:
unk_index , pad_index

(0, 1)

In [21]:
en_vocab.set_default_index(unk_index)
zh_vocab.set_default_index(unk_index)

## To numerical

In [26]:
train_tokens

Dataset({
    features: ['translation', 'en_tokens', 'zh_tokens'],
    num_rows: 231266
})

In [27]:
def to_indices(pair, en_vocab, zh_vocab):
    en_ids = en_vocab.lookup_indices(pair['en_tokens'])
    zh_ids = zh_vocab.lookup_indices(pair['zh_tokens'])
    return {"en_ids": en_ids, "zh_ids": zh_ids}

In [28]:
fn_kwargs = {"en_vocab": en_vocab, "zh_vocab": zh_vocab}

train_tokens = train_tokens.map(to_indices, fn_kwargs=fn_kwargs)
valid_tokens =  valid_tokens.map(to_indices, fn_kwargs=fn_kwargs)
test_tokens =  test_tokens.map(to_indices, fn_kwargs=fn_kwargs)

Map: 100%|██████████| 8549/8549 [00:00<00:00, 11260.21 examples/s]


## Convert to PyTorch tensors to use w pytorch

In [35]:
train_tokens

Dataset({
    features: ['translation', 'en_tokens', 'zh_tokens', 'en_ids', 'zh_ids'],
    num_rows: 231266
})

In [32]:
data_type = "torch" # convert to pytorch tensors 
format_columns = ["en_ids", "zh_ids"]

train_tokens = train_tokens.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_tokens = valid_tokens.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_tokens = test_tokens.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [37]:
train_tokens

Dataset({
    features: ['translation', 'en_tokens', 'zh_tokens', 'en_ids', 'zh_ids'],
    num_rows: 231266
})

In [36]:
train_tokens[1]['en_ids']

tensor([   2,   10,   25,   99, 3954,  252,   60,   17, 1255,    5,   10,   89,
           6,  144,   35,    7,   13,   23,    4,  120,  637,    3])

In [41]:
type(train_tokens[0]["en_ids"])

torch.Tensor

## Data Loaders

In [42]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [pair['en_ids'] for pair in batch]
        batch_zh_ids = [pair['zh_ids'] for pair in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_zh_ids = nn.utils.rnn.pad_sequence(batch_zh_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "zh_ids": batch_zh_ids,
        }
        return batch 
    return collate_fn


In [44]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [45]:
batch_size = 10 #64 #128

train_data_loader = get_data_loader(train_tokens, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_tokens, batch_size, pad_index)
test_data_loader = get_data_loader(test_tokens, batch_size, pad_index)

## Building Model

In [48]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell
    


In [49]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [51]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [52]:
input_dim = len(en_vocab)
output_dim = len(zh_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [53]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(30823, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(45960, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=45960, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [54]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 50,590,344 trainable parameters


In [55]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [56]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["en_ids"].to(device)
        trg = batch["zh_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [57]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["en_ids"].to(device)
            trg = batch["zh_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [58]:
n_epochs = 1
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

100%|██████████| 1/1 [55:39<00:00, 3339.57s/it]

	Train Loss:   5.922 | Train PPL: 373.104
	Valid Loss:   6.093 | Valid PPL: 442.607


In [59]:
model.load_state_dict(torch.load("tut1-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 5.838 | Test PPL: 343.245 |


In [80]:
def translate_sentence(
    sentence,
    model,
    en_vocab,
    zh_vocab,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens =  [token.text for token in nlp.tokenizer(normalizeString(sentence))]
            print(tokens)
        else:
            print("not an instance")
            tokens = [token for token in sentence]
        tokens = [sos_token] + tokens + [eos_token]
        ids = en_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = zh_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == zh_vocab[eos_token]:
                break
        tokens = zh_vocab.lookup_tokens(inputs)
    return tokens

In [66]:
sentence = test_data[0]['translation']['en']
expected_translation = test_data[0]['translation']['zh']

sentence, expected_translation

('Several years ago here at TED, Peter Skillman  introduced a design challenge  called the marshmallow challenge.',
 '几年前，在TED大会上， Peter Skillman 介绍了一个设计挑战 叫做“棉花糖挑战”')

In [82]:
sentence = "how are you?"
translation = translate_sentence(
    sentence,
    model,
    # nlp,
    en_vocab,
    zh_vocab,
    sos_token,
    eos_token,
    device,
)

translation

['how', 'are', 'you', '?']


['<sos>', '你', '什么', '？', '？', '<eos>']